In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!mkdir data
!cp "/content/drive/MyDrive/image_model_X_unflatterned.npy" "data/image_model_X_unflatterned.npy"
!cp "/content/drive/MyDrive/image_model_y.npy" "data/image_model_y.npy"

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torchvision
import torch.nn.functional as F
import numpy as np
from PIL import Image
from torchvision.transforms import transforms
from torch.utils.data import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
import time
import copy
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler
import torch.nn.functional as F
from google.colab import drive
# Load the TensorBoard notebook extension
%reload_ext tensorboard
%tensorboard --logdir runs


X = np.load("data/image_model_X_unflatterned.npy", allow_pickle=True)
y = np.load("data/image_model_y.npy")
# X = pd.DataFrame(X)
# y = pd.DataFrame(y)




In [ ]:
class ProductImageCategoryDataset(Dataset):
    def __init__(self, X, y, transform=None):
        super().__init__()
        self.X = X
        self.y = y
        self.transform = transform
        assert len(self.X) == len(self.y)

    def __getitem__(self, index):
        features = transforms.ToTensor()(self.X[index])
        label = self.y[index]
        if self.transform:
            features = self.transform(features)
        label = int(label)
        return (features, label)

    def __len__(self):
        return len(self.X)




In [ ]:
test_dataset = ProductImageCategoryDataset(X, y)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)

dataiter = iter(test_loader)
images, labels = dataiter.next()

# show images
grid = torchvision.utils.make_grid(images, nrow=4)
img = torchvision.transforms.ToPILImage()(grid)
img.show()


In [ ]:
# Data augmentation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
def train_validation_test_split(X, y, data, train_percent, val_percent, img_transforms, batch_size, shuffle=True):
    data_size = len(data(X, y))
    train_size = int(train_percent * data_size)
    val_size = int(val_percent * data_size)
    test_size = data_size - (train_size + val_size)

    indices = list(range(data_size))
    if shuffle:
      np.random.shuffle(indices)

    train_split_indice = indices[:train_size]
    val_split_indice = indices[train_size:(val_size + train_size)]
    test_split_indice =indices[(val_size + train_size):data_size]

    print(train_split_indice)
    print(len(train_split_indice))
    print(val_split_indice)
    print(len(val_split_indice))
    print(test_split_indice)
    print(len(test_split_indice))


    # Create dictionary of dataset sizes
    dataset_sizes = {"train": train_size,
    "val": val_size,
    "test": test_size
    }

    splits = {"train": train_split_indice,
    "val": val_split_indice,
    "test": test_split_indice
    }

    # SHUFFLE DATA BEFORE SPLIT - Categories in dataset might be in order

    samplers = {"train": SubsetRandomSampler(splits["train"]),
        "val": SubsetRandomSampler(splits["val"]),
        "test": SubsetRandomSampler(splits["test"])
        }

    # train_data, val_data, test_data = torch.utils.data.random_split(data(), [train_size, val_size, test_size])
    
    dataset = {phase: data(X, y, transform=img_transforms[phase])
        for phase in ["train", "val", "test"]}

    data_loader = {phase: DataLoader(dataset[phase], batch_size=batch_size, sampler=samplers[phase])
        for phase in ["train", "val", "test"]}

    return data_loader, dataset_sizes

data_loader, dataset_sizes = train_validation_test_split(X, y, ProductImageCategoryDataset, 0.7, 0.2, data_transforms, 30)

print(data_loader)
print(data_loader['val'])
# for batch in data_loader['val']:
#   print(batch)
print(dataset_sizes)

[222, 3816, 5623, 11443, 10774, 2205, 4942, 9325, 5014, 7298, 1988, 3704, 4432, 10248, 11419, 801, 6408, 2659, 3480, 414, 3524, 7832, 6681, 3606, 726, 2396, 532, 3051, 4181, 4024, 9525, 3779, 5139, 2585, 5626, 8663, 9311, 6607, 9288, 5655, 3654, 8114, 11462, 10921, 8513, 2913, 10797, 5107, 6643, 1605, 7261, 3885, 6482, 6015, 4959, 2887, 5099, 5831, 9446, 9008, 7575, 3929, 5234, 11528, 8779, 992, 9233, 2149, 6067, 2865, 9315, 6994, 2248, 3607, 11009, 11035, 1123, 5023, 3656, 101, 955, 2664, 10054, 727, 5319, 7873, 2434, 7006, 2531, 2727, 3175, 10896, 2889, 3552, 6100, 3442, 4638, 5683, 6597, 4342, 11538, 10820, 3187, 11393, 8166, 5375, 11195, 5778, 8693, 1098, 9613, 2864, 4640, 3089, 3215, 7398, 9857, 5185, 3760, 7466, 1863, 8572, 8084, 11046, 9174, 7754, 9063, 0, 2844, 7543, 8165, 9465, 5127, 3815, 9813, 8833, 3882, 8177, 8895, 5732, 2277, 11432, 11048, 5473, 10599, 2800, 5696, 10558, 9657, 1628, 8809, 10886, 6303, 9385, 1943, 9666, 3343, 5449, 3241, 6163, 11125, 2793, 10431, 3289, 984

In [ ]:
# Loading pretrained model
from torchvision import models

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

class ResNet50(torch.nn.Module):
    def __init__(self, out_size):
        super(ResNet50, self).__init__()
        self.features = models.resnet50(pretrai
        ned=True).to(device) # get the convolutional layers
        for i, param in enumerate(self.features.parameters()):
            if i < 47:
                param.requires_grad=False
            else:
                param.requires_grade=True
        self.features.fc = torch.nn.Sequential(
            torch.nn.Linear(2048, 512), # first arg is the size of the flattened output from 
            torch.nn.ReLU(),
            torch.nn.Dropout(),
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, out_size)
        )

    def forward(self, x):
        # What does this do ?
        x = self.features(x)
        x = x.reshape(x.shape[0], -1)
        return x

model = ResNet50(13).to(device)


Using cuda for inference


In [ ]:
def train_model(model, num_epochs, optimiser, criterion, scheduler, model_load_location=None):
    writer = SummaryWriter()
    
    since = time.time()

    # keep best model at every epoch by copying weights
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        # print current epoch
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                print("Training...")
                model.train()  # Set model to training mode
            else:
                print("Validating...")
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for batch_idx, batch in enumerate(data_loader[phase]):
                features, labels = batch
                features, labels = features.to(device).float(), labels.to(device)

                # zero the parameter gradients
                optimiser.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(features)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # loss = F.cross_entropy(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimiser.step()

                # statistics
                running_loss += loss.item() * features.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()
            print(dataset_sizes[phase])
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            writer.add_scalar('Epoch Loss', epoch_loss, epoch)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == "train":
                writer.add_scalar("Training Accuracy", epoch_acc*100, epoch)
            else:
                writer.add_scalar("Validation Accuracy", epoch_acc*100, epoch)

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

            writer.flush()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)

    # if model_save_location != None:
    #     torch.save(model.state_dict(), model_save_location)

    return model

criterion = nn.CrossEntropyLoss()
optimiser = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimiser, T_0=10, eta_min=0.00001)

model_ft = train_model(model, 50, optimiser, criterion, exp_lr_scheduler)